In [11]:
from os import path
from collections import Counter

import xml
import xml.etree.ElementTree as ET

from transformers import BertTokenizer, LongformerTokenizerFast

In [12]:
import os
os.environ['TRANSFORMERS_OFFLINE']='True'

In [13]:
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-large-4096', add_prefix_space=True)

wsc_file = "/home/shtoshni/Research/coref_resources/data/wsc/WSCollection.xml"
len_limit = 273

tree = ET.parse(wsc_file)
root = tree.getroot()

In [14]:
print(len(list(root))) 
## 12 additional examples have been added to the original 273

285


In [15]:
def search_span(word_list, token_list):
    for start_idx in range(0, len(word_list) - len(token_list) + 1):
        match = start_idx
        for token1, token2 in zip(word_list[start_idx: start_idx + len(token_list)], token_list):
            if token1 != token2:
                match = -1
                break
        
#         print(word_list, token_list, match)

        if match == -1:
            continue
        else:
            return match
        
    return -1

def flatten(l):
    return [item for sublist in l for item in sublist]


In [16]:
    
prefixes = []
pronouns = []
continuations = []

answers = []
correct_answers = []

for elem in list(root)[:len_limit]:
    for children in list(elem.iter('txt1')):
        prefixes.append(children.text.strip().replace('\n', ' '))

    for children in list(elem.iter('pron')):
        pronouns.append(children.text.strip())

    for children in list(elem.iter('txt2')):
        continuations.append(children.text.strip())

    for children in list(elem.iter('answer')):
        answers.append(children.text.strip())


    for children in list(elem.iter('correctAnswer')):
        correct_answers.append(children.text.strip()[0])

    
print(len(answers))
print(len(pronouns))
print(len(prefixes))


print(f'{prefixes[0]} {pronouns[0]} {continuations[0]}')

    

546
546
273
The city councilmen refused the demonstrators a permit because they feared violence.


In [17]:
not_found_count = 0
for idx, prefix in enumerate(prefixes):
    answer1 = answers[idx * 2]
    answer2 = answers[idx * 2 + 1]
    
    text = f'{prefix} {pronouns[idx * 2]} {continuations[idx]}'
    word_list = tokenizer.tokenize(prefix)
    prefix_idx = len(word_list)
    word_list += tokenizer.tokenize(pronouns[idx * 2])
    
    pronoun_boundary = [prefix_idx, len(word_list) - 1]
    word_list += tokenizer.tokenize(continuations[idx])
    
    answer_boundaries = []
    
    for answer in [answer1, answer2]:
        for span in [answer, answer.lower(), answer.capitalize()]:
            span_tokens = tokenizer.tokenize(span)
            found = search_span(word_list, span_tokens)
            if found != -1:
                answer_boundaries.append([[found, found + len(span_tokens) - 1]])
                break
                
        if found == -1:        
            print(text, answer)
            not_found_count += 1
            
    
    import copy
    clusters = copy.deepcopy(answer_boundaries)
    if len(answer_boundaries) == 2:
        correct_answer = correct_answers[idx]
        assert (correct_answer in ['A', 'B'])
        
        if correct_answer == 'A':
            clusters[0].append(pronoun_boundary)
        else:
            clusters[1].append(pronoun_boundary)
        
        
        
    if idx <= 5:
        print(word_list)
        
        print(pronoun_boundary)
        print(answer_boundaries)
        print(clusters)
        
        all_boundaries = flatten(clusters)
        all_boundaries.append(pronoun_boundary)
        
        for ment_start, ment_end in all_boundaries:
            print(word_list[ment_start: ment_end + 1])

    
#     break

print(not_found_count)


['ĠThe', 'Ġcity', 'Ġcouncil', 'men', 'Ġrefused', 'Ġthe', 'Ġdemonstrators', 'Ġa', 'Ġpermit', 'Ġbecause', 'Ġthey', 'Ġfeared', 'Ġviolence', '.']
[10, 10]
[[[0, 3]], [[5, 6]]]
[[[0, 3], [10, 10]], [[5, 6]]]
['ĠThe', 'Ġcity', 'Ġcouncil', 'men']
['Ġthey']
['Ġthe', 'Ġdemonstrators']
['Ġthey']
['ĠThe', 'Ġcity', 'Ġcouncil', 'men', 'Ġrefused', 'Ġthe', 'Ġdemonstrators', 'Ġa', 'Ġpermit', 'Ġbecause', 'Ġthey', 'Ġadvocated', 'Ġviolence', '.']
[10, 10]
[[[0, 3]], [[5, 6]]]
[[[0, 3]], [[5, 6], [10, 10]]]
['ĠThe', 'Ġcity', 'Ġcouncil', 'men']
['Ġthe', 'Ġdemonstrators']
['Ġthey']
['Ġthey']
['ĠThe', 'Ġtrophy', 'Ġdoesn', "'t", 'Ġfit', 'Ġinto', 'Ġthe', 'Ġbrown', 'Ġsuitcase', 'Ġbecause', 'Ġit', 'Ġis', 'Ġtoo', 'Ġlarge', '.']
[10, 10]
[[[0, 1]], [[6, 8]]]
[[[0, 1], [10, 10]], [[6, 8]]]
['ĠThe', 'Ġtrophy']
['Ġit']
['Ġthe', 'Ġbrown', 'Ġsuitcase']
['Ġit']
['ĠThe', 'Ġtrophy', 'Ġdoesn', "'t", 'Ġfit', 'Ġinto', 'Ġthe', 'Ġbrown', 'Ġsuitcase', 'Ġbecause', 'Ġit', 'Ġis', 'Ġtoo', 'Ġsmall', '.']
[10, 10]
[[[0, 1]], [[6, 8]]